# Evaulate Distilbert TopicChooser Model
### 25 Nov 2020

## I. Evaluation Parameters

In [11]:
# Set Evaulation Parameters
MODEL_NAME = "roberta-base"
MODEL_FILE ="saved_model_epoch10_20201129_0424.tar"
VAL_DATASET = "val_data_28Nov_roberta_pf.pickle"
EVAL_BATCH_SIZE = 16
DESCRIPTION = "PolitiFact only model"

## II. Setup and Load Model

In [2]:
import pickle
import sys

import json
import pandas as pd
import pickle
import sklearn.metrics
import tqdm
import torch
import transformers as hft

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [3]:
# Load evaluation data
with open(VAL_DATASET, "rb") as vfile:
    val_dataset = pickle.load(vfile)

In [4]:
# Load saved model
checkpoint = torch.load(MODEL_FILE)
model = (hft.RobertaForSequenceClassification
         .from_pretrained(MODEL_NAME))
model.load_state_dict(checkpoint["model"])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

<All keys matched successfully>

In [5]:
# Move model to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
model.to(device)
model.eval();
device

device(type='cuda')

## III. Evaluation Loop and Results Dataframe

In [6]:
# Evaulate model
eval_loader = torch.utils.data.DataLoader(val_dataset,
                                batch_size=EVAL_BATCH_SIZE,
                                shuffle=False)
labels = []
preds = []
logits = []
probs = []
sources = []
article_tokens = []
file_names = []

softmax = torch.nn.Softmax(dim=1)

with torch.no_grad():
    for art_num, article in tqdm.tqdm(enumerate(eval_loader)):
        input_ids = article["input_ids"].to(device)
        attent_mask = article["attention_mask"].to(device)
        output = model(input_ids,
                       attention_mask=attent_mask,
                       output_hidden_states=False,
                       output_attentions=False)
        prob = softmax(output[0].detach().cpu()).numpy()
        logit = output[0].detach().cpu().numpy()
        pred = [0 if lgt[0] > lgt[1] else 1 for lgt in logit]
        label = article["labels"].numpy()
        labels.extend(label)
        preds.extend(pred)
        logits.extend(logit)
        probs.extend(prob)
        article_tokens.extend(article["article_tokens"].numpy())
        sources.extend(article["sources"])
        file_names.extend(article["file_names"])

3it [00:02,  1.17it/s]


In [7]:
# Label of 0 is real, 1 is fake
eval_results = pd.DataFrame({"Article": list(range(len(labels))),
                             "Source": sources,
                             "Token_Length": article_tokens,
                             "Predictions": preds,
                             "Labels": labels,
                             "Logit_real(0)": [x[0] for x in logits],
                             "Logit_fake(1)": [x[1] for x in logits],
                             "Prob_real(0)": [x[0] for x in probs],
                             "Prob_real(1)": [x[1] for x in probs],
                             "File_Name": file_names})

eval_results

,Article,Source,Token_Length,Predictions,Labels,Logit_real(0),Logit_fake(1),Prob_real(0),Prob_real(1),File_Name
0,0,politifact,10104,0,0,4.232014,-4.277524,0.999798,0.000201,politifact304
1,1,politifact,246,1,1,-4.695490,4.622222,0.000090,0.999910,politifact13663
2,2,politifact,330,1,1,-4.474310,4.359578,0.000146,0.999854,politifact13823
3,3,politifact,360,1,1,-4.668920,4.603256,0.000094,0.999906,politifact14447
4,4,politifact,481,1,0,-2.317299,1.872538,0.014923,0.985077,politifact2258
...,...,...,...,...,...,...,...,...,...,...
130,130,politifact,250,1,0,-3.785406,3.608128,0.000615,0.999385,politifact13352
131,131,politifact,793,1,1,-4.607570,4.474548,0.000114,0.999886,politifact15241
132,132,politifact,4956,0,0,4.005533,-4.207335,0.999729,0.000271,politifact51
133,133,politifact,312,1,1,-4.122876,3.760148,0.000377,0.999623,politifact14888


In [8]:
# Save evaluation data to csv file
eval_results.to_csv("eval_data_" + MODEL_FILE[12:-4] + ".csv")

## IV. Overall Metrics

In [12]:
# Overall metrics
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
    eval_results.Labels, eval_results.Predictions, average="binary")
accuracy = sklearn.metrics.accuracy_score(eval_results.Labels, eval_results.Predictions)

metrics = {"description": DESCRIPTION,
           "eval_notes": "Evaluating PolitiFact data",
           "model": MODEL_FILE,
           "epochs": checkpoint["epoch"],
           "batch_size": EVAL_BATCH_SIZE,
           "Date_evaluated": "28Nov2020",
           "train_data": "train_data_28Nov_roberta_pf.pickle",
           "val_data": "eval_data_" + MODEL_FILE[12:-4] + ".csv",
           "eval_data": VAL_DATASET,
           "accuracy": accuracy,
           "precision": precision,
           "recall": recall,
           "f1": f1}
metrics

{'description': 'PolitiFact only model',
 'eval_notes': 'Evaluating PolitiFact data',
 'model': 'saved_model_epoch10_20201129_0424.tar',
 'epochs': 10,
 'batch_size': 16,
 'Date_evaluated': '28Nov2020',
 'train_data': 'train_data_28Nov_roberta_pf.pickle',
 'val_data': 'eval_data_epoch10_20201129_0424.csv',
 'eval_data': 'val_data_28Nov_roberta_pf.pickle',
 'accuracy': 0.9555555555555556,
 'precision': 0.922077922077922,
 'recall': 1.0,
 'f1': 0.9594594594594594}

In [13]:
# Save metrics to JSON text file.
with open("eval_metrics_" + MODEL_FILE[12:-4] + ".json", "wt") as jfile:
    json.dump(metrics, jfile)

In [13]:
cm = pd.DataFrame(sklearn.metrics.confusion_matrix(eval_results.Labels, eval_results.Predictions))
label_titles = {0: "gossip", 1: "political"}
cm.rename(index=label_titles, columns=label_titles, inplace=True)
cm

,gossip,political
gossip,2050,616
political,135,727
